In [1]:
import warnings
import os
import sys

warnings.filterwarnings('ignore')
current_dir = %pwd

parent_dir = os.path.abspath(os.path.join(current_dir, '../..'))
sys.path.append(parent_dir)

In [2]:
from src.main.models.bidirLSMT import BidirectionalLSTM
from src.main.utilities.utils import clean_text, stop_words_removal, get_dataset
from sklearn.model_selection import train_test_split

import warnings
warnings.filterwarnings("ignore")

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/francesco/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [3]:
 # define model and pipeline
bidirLSTM = BidirectionalLSTM(vocab_size=100000, embedding_dim=300, max_sequence_length=100, lstm_units=10)
bidirLSTM.pipeline = [stop_words_removal]
# import dataset and apply pipeline
inputs, targets = get_dataset(one_hot=True)

inputs = bidirLSTM.run_pipeline(inputs, save=True)

# stratified train test split
x_train, x_test, y_train, y_test = train_test_split(inputs, targets, test_size=0.2, random_state=42, stratify=targets)

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/francesco/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/francesco/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/francesco/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/francesco/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/francesco/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/francesco/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/francesco/nltk_data...
[nltk_data]   Package stopwords i

Pipeline execution time: 0:00:04.642869


In [4]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

def tokenize(sequences):
    # Tokenization
    tokenizer = Tokenizer()
    tokenizer.fit_on_texts(sequences)
    sequences_as_integers = tokenizer.texts_to_sequences(sequences)

    # Padding
    max_sequence_length = 100  # Choose an appropriate value based on your data
    return pad_sequences(sequences_as_integers, maxlen=max_sequence_length)

In [5]:
x_train = tokenize(x_train)
x_test = tokenize(x_test)

In [6]:
bidirLSTM.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ (None, None, 300)      │    30,000,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional (Bidirectional)   │ (None, None, 20)       │        24,880 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional_1 (Bidirectional) │ (None, 20)             │         2,480 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 5)              │           105 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 30,027,465 (114.55 MB)

 Trainable params: 30,027,465 (114.55 MB)

 Non-trainable params: 0 (0.00 B)

None


In [7]:
bidirLSTM.fit(x_train, y_train, batch_size=300, epochs=3, validation_data=(x_test, y_test), verbose=1)

Epoch 1/3
289/289 ━━━━━━━━━━━━━━━━━━━━ 76s 257ms/step - accuracy: 0.6071 - loss: 1.0713 - val_accuracy: 0.3740 - val_loss: 1.8537
Epoch 2/3
289/289 ━━━━━━━━━━━━━━━━━━━━ 77s 267ms/step - accuracy: 0.8686 - loss: 0.4117 - val_accuracy: 0.3665 - val_loss: 2.1398
Epoch 3/3
289/289 ━━━━━━━━━━━━━━━━━━━━ 76s 264ms/step - accuracy: 0.9418 - loss: 0.2146 - val_accuracy: 0.3554 - val_loss: 2.4146


In [8]:
%load_ext tensorboard
%tensorboard --logdir {config.TENSORBOARD_LOGS.format(repr(bidirLSTM))} --port=6969

In [9]:
bidirLSTM.evaluate(x_test, y_test)

677/677 ━━━━━━━━━━━━━━━━━━━━ 6s 9ms/step - accuracy: 0.3565 - loss: 2.4113


[2.414567232131958, 0.3553676903247833]

In [10]:
from src.main.utilities import plotting

y_pred = bidirLSTM.predict(x_test)
results = plotting.compute_performance(y_test, y_pred)
plotting.print_performance(results)


677/677 ━━━━━━━━━━━━━━━━━━━━ 7s 10ms/step


ValueError: Classification metrics can't handle a mix of multilabel-indicator and continuous-multioutput targets